Hello Guys.
In this Notebook we are going to implement a music Genre classification using K-Nearest Neighbors Classifier algorithm. I hope the notebook will be helpful for the begineers starting with Machine Learning as well with the Audio Processing using Machine Learning.

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [6]:
!pip install python_speech_features

^C


In [3]:
#import libraries
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

ModuleNotFoundError: No module named 'python_speech_features'

In [ ]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [ ]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

* trainingset is actual training data and k is hyperparameter of how much nearest neighbors we want.
* first part itera gets distances of alll the points in training set
* we sort the data and get the nearest neighbor, as the data is sorted so it will easily take the k nearest neighbors. and we finally return those neighbors

In [ ]:
#function to identify the nearest neighbors
def nearestclass(neighbors):
    classVote = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

* here we define a set, which will help to take the reponse of each class present in nearest neighbors.
* actually the idea behind this function is that, we already have a list of nearest neighbors and as data is labelled data so each neighbor will hold some particular class, now from that nearest neighbor whatever the maximum frequency will be there for each class, to that class the new data will be assigned.
* eg- k=10, so will have the list of 10 nearest neighbor, suppose(Class A=4, B=3, C=3) so new point will be assigned to A as of maximum frequency. 

In [ ]:
# define a function that will evaluate a model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

In [ ]:
import librosa, IPython
import librosa.display
file = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00031.wav'
signal, sr = librosa.load(file , sr = 22050) 
IPython.display.Audio(signal, rate=sr)

In [ ]:
directory = '../input/gtzan-dataset-music-genre-classification/Data/genres_original'
f = open("my.dat", "wb")
i = 0
for folder in os.listdir(directory):
    #print(folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+"/"+folder):
        #print(file)
        try:
            (rate, sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
f.close()

* after the cell run we will be having a my.dat file as a collection of different files irrespective of a jonar.
* we pich each file from genre and calculate rate and signature.
* what we have done is, we find the imp feature as mfcc(mel frequency celestral coefficients) then we find the feature and then we dump the feature into my.dat file.
* Now we have a dataset, so we will split into train and test set

In [ ]:
#split dataset into train and test set
dataset = []

def loadDataset(filename, split, trset, teset):
    with open('my.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

* we will load the dataset again to split it, we take 75% as train and 25% as test data.
* we split the dataset based on split value, because directly splitting will not give as good result because we want the random sample which is the mix of all the jonars.

In [ ]:
# Make the prediction using KNN(K nearest Neighbors)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

In [ ]:
from collections import defaultdict
results = defaultdict(int)

directory = "../input/gtzan-dataset-music-genre-classification/Data/genres_original"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
pred = nearestclass(getNeighbors(dataset, feature, 5))
print(results[pred])

## End Note

Processing Audio Data is way more complex then the image processing at current scenarios. But Python has a great libraries by the help of which we can extract the audio features and analyze them on different scale as time domain. So I hope you like this particular implementation.

Please If I was able to help you a little bit then upvote this Notebook. Its to much appreciating for me to move forward in my data science carrer.

**Thank You!..**